# Static Replication of European Payoffs

## Import

### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import datetime as dt

from scipy.stats import norm
from scipy.optimize import brentq
from scipy.optimize import least_squares
from scipy.integrate import quad

import warnings
warnings.filterwarnings('ignore')

### Parameters (Calibrated SABR Model)

In [6]:
# Calibrated parameters saved to an external csv file for easy retrieval
# Avoid issue of having to always run the model calibration cells
# SABR parameters were stored in the following order: alpha, beta, rho, nu
calibrated_params = pd.read_csv("Calibrated_Model_params.csv")

static_vol_SPX = calibrated_params["SPX_ATM_Vols"].values[0]
static_vol_SPY = calibrated_params["SPY_ATM_Vols"].values[0]

SABR_alpha_SPX = calibrated_params["SABR_SPX"].values[0]
SABR_beta_SPX = calibrated_params["SABR_SPX"].values[1]
SABR_rho_SPX = calibrated_params["SABR_SPX"].values[2]
SABR_nu_SPX = calibrated_params["SABR_SPX"].values[3]

SABR_alpha_SPY = calibrated_params["SABR_SPY"].values[0]
SABR_beta_SPY = calibrated_params["SABR_SPY"].values[1]
SABR_rho_SPY = calibrated_params["SABR_SPY"].values[2]
SABR_nu_SPY = calibrated_params["SABR_SPY"].values[3]

### Data

In [8]:
discount_rates = pd.read_csv("zero_rates_20201201.csv")
discount_rates["date"] = pd.to_datetime(discount_rates["date"], format = "%Y%m%d")
min = discount_rates["days"].min()
max = discount_rates["days"].max()
discount_rates = discount_rates.set_index("days").reindex(index = list(range(min, max+1)))
discount_rates["rate"] = discount_rates["rate"].interpolate(method='index', limit_direction='forward')
discount_rates["rate"] /= 100
discount_rates = discount_rates.ffill().reset_index()

In [9]:
curr_date = dt.datetime(2020, 12, 1)
maturity_date = dt.datetime(2021, 1, 15)
static_T = (maturity_date - curr_date).days
static_r = discount_rates.iloc[static_T - 7]["rate"]

In [11]:
SPX_index_val = 3662.45

## Payoff function

Suppose on 1-Dec-2020, there is a need to evaluate an exotic European derivative expiring on 15-Jan-2021 which pays:
$$S_{T}^{1/3} + 1.5\log S_{T} + 10.0$$

### Black-Scholes Model

Pricing function based on Black-Scholes Model is given by:
$$V_{0} = S_{0}^{\frac{1}{3}}\Biggl[e^{(-2r-\frac{\sigma^2}{3})\frac{T}{3}}\Biggr] + 1.5\Biggl[\log(S_{0} + (r - \frac{\sigma^2}{2})T\Biggr] + 10$$

In [12]:
def payoffBlackScholes(S, r, T, sigma):
    price = (S ** (1/3)) * (np.exp((T/3) * (-2*r - ((sigma**2)/3)))) + (1.5*np.log(S)) + (1.5*T*(r - (sigma**2)/2)) + 10
    return price

In [13]:
price_derivative_BS_SPX = payoffBlackScholes(SPX_index_val, static_r, static_T/365, static_vol_SPX)

print(f"Price of derivative contract for SPX (Black Scholes model): ${price_derivative_BS_SPX:.5f}")

Price of derivative contract for SPX (Black Scholes model): $37.71009


### Bachelier Model

Pricing function based on Bachelier Model is given by:
$$V_{0} = \frac{e^{-rT}}{\sqrt{2\pi}}\int_{-\infty}^{\infty}\Biggl[(S_{0}+\sigma\sqrt{T}x)^\frac{1}{3} + 1.5\log(S_{0}+\sigma\sqrt{T}x) +\
10\Biggr]\cdot e^{-\frac{x^2}{2}} dx$$

In [14]:
def payoffBachelier(S, r, T, sigma):
    integrand = lambda x: (((S + sigma*np.sqrt(T)*x)**(1/3) + (1.5*np.log(S + sigma*np.sqrt(T)*x)) + 10) * np.exp(-(x**2)/2))
    price = np.exp(-r*T)/np.sqrt(2*np.pi) * quad(integrand, -(np.inf), np.inf)[0]
    return price

In [15]:
price_derivative_Bach_SPX = payoffBachelier(SPX_index_val, static_r, static_T/365, static_vol_SPX)

print(f"Price of derivative contract for SPX (Bachelier model): ${price_derivative_Bach_SPX:.5f}")

Price of derivative contract for SPX (Bachelier model): $37.71360


### Static-replication of European payoff (using calibrated SABR Model)

Static replication of payoff is given by:

$$V_{0} = e^{-rT}((S_{0}e^{rT})^{\frac{1}{3}} + 1.5\log(S_{0}e^{rT}) + 10) +\
\int_{0}^{F}(-\frac{2}{9}K^{-\frac{5}{3}} - \frac{1.5}{K^{2}})\cdot P(K) dK +\
\int_{F}^{\infty}(-\frac{2}{9}K^{-\frac{5}{3}} - \frac{1.5}{K^{2}})\cdot C(K) dK$$

In [19]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2)/24) * alpha * alpha/(F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha/(F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho)/24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T)/(F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha) * ((F * X) ** (0.5 * (1-beta))) * np.log(F/X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta) ** 2)/24) * ((alpha * alpha)/((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha/((F * X) ** ((1 - beta)/2))
        numer3 = ((2 - 3 * rho * rho)/24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2/24) * (np.log(F/X)) ** 2
        denom2 = (((1 - beta) ** 4)/1920) * ((np.log(F/X)) ** 4)
        denom = ((F*X) ** ((1 - beta)/2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer/denom

    return sabrsigma

In [20]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, sabr_vol, K, r, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, sabr_vol, K, r, T)


def sabrcallintegrand_payoff(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * (-2/9*K**(-5/3) - 1.5/K**2)
    return price


def sabrputintegrand_payoff(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * (-2/9*K**(-5/3) - 1.5/K**2)
    return price

In [21]:
def payoffSABR(S, r, T, alpha, beta, rho, nu):
    F = S*np.exp(r*T)
    I_put = quad(lambda x: sabrputintegrand_payoff(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
    I_call = quad(lambda x: sabrcallintegrand_payoff(x, S, r, T, alpha, beta, rho, nu), F, np.inf)
    price = (np.exp(-r*T) * (F**(1/3) + 1.5*np.log(F) + 10)) + (I_put[0] + I_call[0])
    return price

In [22]:
price_derivative_SABR_SPX = payoffSABR(SPX_index_val,
                                       static_r,
                                       static_T/365,
                                       SABR_alpha_SPX,
                                       SABR_beta_SPX,
                                       SABR_rho_SPX,
                                       SABR_nu_SPX)

print(f"Price of derivative contract for SPX (SABR model): ${price_derivative_SABR_SPX:.5f}")

Price of derivative contract for SPX (SABR model): $37.70037


## "Model-free" integrated variance

Suppose on 1-Dec-2020, there is a need to evaluate an exotic European derivative expiring on 15-Jan-2021 which pays the "Model-free" integrated variance:
$$\sigma_{MF}^{2}T = \mathbb{E}\Biggl[\int_{0}^{T}\sigma_{t}^{2}\ dt\Biggr] = 2e^{rT}\int_{0}^{F}\frac{P(K)}{K^2}\ dK + 2e^{rT}\int_{F}^{\infty}\frac{C(K)}{K^2}\ dK$$

### Black-Scholes Model

In [2]:
def BlackScholesCall(S, sigma, K, r, T):
    d1 = (np.log(S/K) + (r+0.5*(sigma**2))*T) / (np.sqrt(T) * sigma)
    d2 = d1 - np.sqrt(T) * sigma
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [3]:
def BlackScholesPut(S, sigma, K, r, T):
    d1 = (np.log(S/K) + (r+0.5*(sigma**2))*T) / (np.sqrt(T) * sigma)
    d2 = d1 - np.sqrt(T) * sigma
    return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [23]:
def BScallintegrand(S, sigma, K, r, T):
    price = BlackScholesCall(S, sigma, K, r, T) / K**2
    return price

def BSputintegrand(S, sigma, K, r, T):
    price = BlackScholesPut(S, sigma, K, r, T) / K**2
    return price

In [24]:
def varianceBlackScholes(S, sigma, r, T):
    F = S*np.exp(r*T)
    I_put = quad(lambda x: BSputintegrand(S, sigma, x, r, T), 1e-6, F)
    I_call = quad(lambda x: BScallintegrand(S, sigma, x, r, T), F, np.inf)
    E_var = (2*np.exp(r*T)/T) * (I_put[0] + I_call[0])
    return E_var

In [25]:
model_free_var_BS_SPX = varianceBlackScholes(SPX_index_val, static_vol_SPX, static_r, static_T/365)

print(f"Fair variance swap strike contract for SPX (Black-Scholes model): {model_free_var_BS_SPX:.5f}")

Fair variance swap strike contract for SPX (Black-Scholes model): 0.03567


### Bachelier Model

In [4]:
# Bachelier volatility multiply by S because it is measured in absolute terms
# While Black-Scholes is measured in percentage
def BachelierCall(S, sigma, K, r, T):
    d1 = (S-K) / (sigma*S*np.sqrt(T))
    return np.exp(-r*T) * ((S-K)*norm.cdf(d1) + sigma*S*np.sqrt(T)*norm.pdf(d1))

In [5]:
def BachelierPut(S, sigma, K, r, T):
    d1 = (S-K) / (sigma*S*np.sqrt(T))
    return np.exp(-r*T) * ((K-S) * norm.cdf(-d1) + sigma*S*np.sqrt(T)*norm.pdf(-d1))

In [26]:
def bacheliercallintegrand(S, sigma, K, r, T):
    price = BachelierCall(S, sigma, K, r, T) / K**2
    return price

def bachelierputintegrand(S, sigma, K, r, T):
    price = BachelierPut(S, sigma, K, r, T) / K**2
    return price

In [27]:
def varianceBachelier(S, sigma, r, T):
    F = S*np.exp(r*T)
    I_put = quad(lambda x: bachelierputintegrand(S, sigma, x, r, T), 1e-6, F)
    I_call = quad(lambda x: bacheliercallintegrand(S, sigma, x, r, T), F, np.inf)
    E_var = (2*np.exp(r*T)/T) * (I_put[0] + I_call[0])
    return E_var

In [28]:
model_free_var_Bach_SPX = varianceBachelier(SPX_index_val, static_vol_SPX, static_r, static_T/365)

print(f"Fair variance swap strike contract for SPX (Bachelier model): {model_free_var_Bach_SPX:.5f}")

Fair variance swap strike contract for SPX (Bachelier model): 0.03591


### Static-replication of European payoff (using calibrated SABR Model)

In [29]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, sabr_vol, K, r, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, sabr_vol, K, r, T)


def sabrcallintegrand_variance(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand_variance(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [30]:
def varianceSABR(S, r, T, alpha, beta, rho, nu):
    F = S*np.exp(r*T)
    I_put = quad(lambda x: sabrputintegrand_variance(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
    I_call = quad(lambda x: sabrcallintegrand_variance(x, S, r, T, alpha, beta, rho, nu), F, np.inf)
    E_var = (2*np.exp(r*T)/T) * (I_put[0] + I_call[0])
    return E_var

In [31]:
model_free_var_SABR_SPX = varianceSABR(SPX_index_val,
                                       static_r,
                                       static_T/365,
                                       SABR_alpha_SPX,
                                       SABR_beta_SPX,
                                       SABR_rho_SPX,
                                       SABR_nu_SPX)

print(f"Fair variance swap strike contract for SPX (SABR model): {model_free_var_SABR_SPX:.5f}")

Fair variance swap strike contract for SPX (SABR model): 0.05151
